In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# CycleGAN 레포 클론
!git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix
%cd pytorch-CycleGAN-and-pix2pix
!pip install -r requirements.txt

Cloning into 'pytorch-CycleGAN-and-pix2pix'...
remote: Enumerating objects: 2516, done.
remote: Total 2516 (delta 0), reused 0 (delta 0), pack-reused 2516 (from 1)
Receiving objects: 100% (2516/2516), 8.20 MiB | 11.91 MiB/s, done.
Resolving deltas: 100% (1575/1575), done.
/content/pytorch-CycleGAN-and-pix2pix
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 63.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 126.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 97.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.6 MB/s eta 0:00:00
   ━━━━━━

In [ ]:
# monet2photo pretrained weights 받기
!bash ./scripts/download_cyclegan_model.sh monet2photo

# fine-tuning용 새 체크포인트 디렉터리 생성
!mkdir -p checkpoints/syn2real_monet_ft
!cp checkpoints/monet2photo_pretrained/latest_net_G.pth checkpoints/syn2real_monet_ft/
!cp checkpoints/monet2photo_pretrained/latest_net_D.pth checkpoints/syn2real_monet_ft/

Note: available models are apple2orange, orange2apple, summer2winter_yosemite, winter2summer_yosemite, horse2zebra, zebra2horse, monet2photo, style_monet, style_cezanne, style_ukiyoe, style_vangogh, sat2map, map2sat, cityscapes_photo2label, cityscapes_label2photo, facades_photo2label, facades_label2photo, iphone2dslr_flower
Specified [monet2photo]
for details.

--2025-05-06 01:40:42--  http://efrosgans.eecs.berkeley.edu/cyclegan/pretrained_models/monet2photo.pth
Resolving efrosgans.eecs.berkeley.edu (efrosgans.eecs.berkeley.edu)... 128.32.244.190
Connecting to efrosgans.eecs.berkeley.edu (efrosgans.eecs.berkeley.edu)|128.32.244.190|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45575747 (43M)
Saving to: ‘./checkpoints/monet2photo_pretrained/latest_net_G.pth’

./checkpoints/monet 100%[===================>]  43.46M  12.7MB/s    in 3.8s    

2025-05-06 01:40:46 (11.6 MB/s) - ‘./checkpoints/monet2photo_pretrained/latest_net_G.pth’ saved [45575747/45575747]

cp: ca

## 데이터셋 경로 셋팅
- CycleGAN은 trainA, trainB, testA 구조 필요
- `trainA`: Synthetic, `trainB`: Real, `testA`: 변환 대상
- Drive 안의 데이터를 symbolic link로 연결

In [ ]:
%%bash
# 1) dataroot 폴더 초기화
BASE=datasets/syn2real
rm -rf $BASE      # 혹시 예전 설정이 꼬여 있다면 삭제
mkdir -p $BASE/{trainA,trainB,testA}

# 2) trainA ← synthetic 이미지
ln -s /content/drive/MyDrive/SynToReal/data/train_total/images/* $BASE/trainA/

# 3) trainB ← real 이미지 (테스트용)
ln -s /content/drive/MyDrive/SynToReal/data/test/images/* $BASE/trainB/

# 4) testA ← real 이미지 (테스트 입력)
ln -s /content/drive/MyDrive/SynToReal/data/test/images/* $BASE/testA/


In [ ]:
!BASE=datasets/syn2real

In [ ]:
# 1) 기존 testA 폴더 통째로 삭제
# !rm -rf /content/pytorch-CycleGAN-and-pix2pix/datasets/syn2real/testA

# 2) 깨끗한 testA 폴더 생성
!mkdir -p /content/pytorch-CycleGAN-and-pix2pix/datasets/syn2real/testA

!ln -s /content/drive/MyDrive/SynToReal/data/train_total/images/*  /content/pytorch-CycleGAN-and-pix2pix/datasets/syn2real/testA/

## Fine-tuning 데이터셋 구성

In [ ]:
# fine_tune_data 폴더 생성
!BASE=/content/drive/MyDrive/SynToReal/fine_tune_data

!mkdir -p $BASE/trainA $BASE/trainB $BASE/testA

# trainA (원본) ← train_total + val_total
!ln -s /content/drive/MyDrive/SynToReal/data/train_total/images/* $BASE/trainA/
!ln -s /content/drive/MyDrive/SynToReal/data/val_total/images/*   $BASE/trainA/

# trainB (스타일) ← test
!ln -s /content/drive/MyDrive/SynToReal/data/test/images/*   $BASE/trainB/

# testA (테스트에 사용할 입력)
!ln -s /content/drive/MyDrive/SynToReal/data/test/images/*        $BASE/testA/


CycleGAN의 continue_train 모드에서는 아래 네 개의 파일이 반드시 있어야 한다:
```
latest_net_G_A.pth
latest_net_G_B.pth
latest_net_D_A.pth
latest_net_D_B.pth
```



In [ ]:
%%bash
# 원본(pretrained) 폴더와 fine-tune 폴더 변수
PRE=checkpoints/monet2photo_pretrained
FT=/content/drive/MyDrive/SynToReal/cyclegan/checkpoints/syn2real_monet_ft/

mkdir -p $FT

# Generator 복사 (A→B, B→A)
cp $PRE/latest_net_G_A.pth $FT/ 2>/dev/null || cp $PRE/latest_net_G.pth $FT/latest_net_G_A.pth
cp $PRE/latest_net_G_B.pth $FT/ 2>/dev/null || cp $PRE/latest_net_G.pth $FT/latest_net_G_B.pth

# Discriminator 복사 (A, B)
cp $PRE/latest_net_D_A.pth $FT/ 2>/dev/null || cp $PRE/latest_net_D.pth $FT/latest_net_D_A.pth
cp $PRE/latest_net_D_B.pth $FT/ 2>/dev/null || cp $PRE/latest_net_D.pth $FT/latest_net_D_B.pth

# 확인
ls -1 $FT


latest_net_G_A.pth
latest_net_G_B.pth
train_opt.txt


cp: cannot stat 'checkpoints/monet2photo_pretrained/latest_net_D.pth': No such file or directory
cp: cannot stat 'checkpoints/monet2photo_pretrained/latest_net_D.pth': No such file or directory


In [ ]:
!python train.py \
  --dataroot datasets/syn2real \
  --name syn2real_monet_ft \
  --model cycle_gan \
  --n_epochs 20 \
  --lr 0.0002 \
  --gpu_ids 0 \
  --checkpoints_dir /content/drive/MyDrive/SynToReal/cyclegan/checkpoints2 \
  --display_freq 200 \
  --save_epoch_freq 5


----------------- Options ---------------
               batch_size: 1                             
                    beta1: 0.5                           
          checkpoints_dir: /content/drive/MyDrive/SynToReal/cyclegan/checkpoints2	[default: ./checkpoints]
           continue_train: False                         
                crop_size: 256                           
                 dataroot: datasets/syn2real             	[default: None]
             dataset_mode: unaligned                     
                direction: AtoB                          
              display_env: main                          
             display_freq: 200                           	[default: 400]
               display_id: 1                             
            display_ncols: 4                             
             display_port: 8097                          
           display_server: http://localhost              
          display_winsize: 256                           
        

KeyboardInterrupt: 

In [ ]:
!python test.py \
  --dataroot datasets/syn2real \
  --name syn2real_monet_ft \
  --model test \
  --no_dropout \
  --gpu_ids 0 \
  --results_dir /content/drive/MyDrive/SynToReal/results/train \
  --num_test 10000

----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: ./checkpoints                 
                crop_size: 256                           
                 dataroot: datasets/syn2real             	[default: None]
             dataset_mode: single                        
                direction: AtoB                          
          display_winsize: 256                           
                    epoch: latest                        
                     eval: False                         
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 3                             
                  isTrain: False                         	[default: None]
                load_iter: 0                            

- checkpoints/syn2real_cyclegan/latest_net_G_A.pth   ← A → B (합성 → 실제)
- checkpoints/syn2real_cyclegan/latest_net_G_B.pth   ← B → A (실제 → 합성)

In [ ]:
# Drive에 학습된 모델 저장
!mkdir -p /content/drive/MyDrive/SynToReal/cyclegan_weights
!cp checkpoints/syn2real_cyclegan/*.pth /content/drive/MyDrive/SynToReal/cyclegan_weights/

In [ ]:
# 변환 대상 이미지 준비 (Falcon 합성 이미지)
!mkdir -p datasets/syn2real/testA
!ln -s "/content/drive/MyDrive/SynToReal/data/train_total/images" datasets/syn2real/testA/images

In [ ]:
!bash ./scripts/download_cyclegan_model.sh monet2photo
!python test.py \
  --name monet2photo_pretrained \
  --model test \
  --no_dropout \
  --gpu_ids 0


Note: available models are apple2orange, orange2apple, summer2winter_yosemite, winter2summer_yosemite, horse2zebra, zebra2horse, monet2photo, style_monet, style_cezanne, style_ukiyoe, style_vangogh, sat2map, map2sat, cityscapes_photo2label, cityscapes_label2photo, facades_photo2label, facades_label2photo, iphone2dslr_flower
Specified [monet2photo]
for details.

--2025-05-06 00:54:16--  http://efrosgans.eecs.berkeley.edu/cyclegan/pretrained_models/monet2photo.pth
Resolving efrosgans.eecs.berkeley.edu (efrosgans.eecs.berkeley.edu)... 128.32.244.190
Connecting to efrosgans.eecs.berkeley.edu (efrosgans.eecs.berkeley.edu)|128.32.244.190|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45575747 (43M)
Saving to: ‘./checkpoints/monet2photo_pretrained/latest_net_G.pth’

./checkpoints/monet 100%[===================>]  43.46M  53.4MB/s    in 0.8s    

2025-05-06 00:54:17 (53.4 MB/s) - ‘./checkpoints/monet2photo_pretrained/latest_net_G.pth’ saved [45575747/45575747]

------

In [ ]:
# 학습된 모델로 testA 이미지 스타일 변환
!python test.py \
  --dataroot ./datasets/syn2real \
  --name monet2photo_pretrained \
  --model test \
  --no_dropout \
  --gpu_ids 0


----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: ./checkpoints                 
                crop_size: 256                           
                 dataroot: ./datasets/syn2real           	[default: None]
             dataset_mode: single                        
                direction: AtoB                          
          display_winsize: 256                           
                    epoch: latest                        
                     eval: False                         
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 3                             
                  isTrain: False                         	[default: None]
                load_iter: 0                            

In [ ]:
# 변환된 이미지 Drive로 복사 (YOLO 학습용)
!mkdir -p /content/drive/MyDrive/SynToReal/yolo_dataset/images/train
!cp results/syn2real_cyclegan/test_latest/images/*.png /content/drive/MyDrive/SynToReal/yolo_dataset/images/train/

# 기존 라벨 복사
!mkdir -p /content/drive/MyDrive/SynToReal/yolo_dataset/labels/train
!cp /content/drive/MyDrive/SynToReal/data/train_total/labels/*.txt /content/drive/MyDrive/SynToReal/yolo_dataset/labels/train/


## 저장된 weight 파일을 CycleGAN 변환(inference)

In [ ]:
!mkdir -p checkpoints/syn2real_cyclegan
!cp /content/drive/MyDrive/SynToReal/cyclegan_weights/latest_net_G_*.pth /content/pytorch-CycleGAN-and-pix2pix/checkpoints/syn2real_cyclegan/

In [ ]:
# 테스트 이미지 준비 (변환 대상)
!mkdir -p datasets/syn2real/testA
!ln -s "/content/drive/MyDrive/SynToReal/data/train_total/images" datasets/syn2real/testA/images

In [ ]:
!python test.py \
  --dataroot ./datasets/syn2real \
  --name syn2real_cyclegan \
  --model test \
  --epoch latest \
  --no_dropout \
  --gpu_ids 0

In [ ]:
# 저장된 모델로 inference
!python /content/pytorch-CycleGAN-and-pix2pix/test.py \
  --dataroot /content/datasets/syn2real \
  --name syn2real_cyclegan \
  --model test \
  --epoch latest \
  --no_dropout \
  --gpu_ids 0

In [ ]:
# 변환 이미지 확인
from PIL import Image
import matplotlib.pyplot as plt

original = Image.open('datasets/syn2real/testA_single/00001.png')
converted = Image.open('results/syn2real_cyclegan/test_latest/images/00001_fake_B.png')

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.title("Original (Synthetic)")
plt.imshow(original)
plt.axis('off')

plt.subplot(1, 2, 2)
plt.title("CycleGAN Converted (Real-style)")
plt.imshow(converted)
plt.axis('off')
plt.show()


In [ ]:
!mkdir -p datasets/syn2real/testA_single
!cp /content/drive/MyDrive/SynToReal/data/train_total/images/00001.png datasets/syn2real/testA_single/


In [ ]:
# 결과를 YOLO 학습용으로 저장
!mkdir -p /content/drive/MyDrive/SynToReal/yolo_dataset/images/train
!cp results/syn2real_cyclegan/test_latest/images/*.png /content/drive/MyDrive/SynToReal/yolo_dataset/images/train/

!mkdir -p /content/drive/MyDrive/SynToReal/yolo_dataset/labels/train
!cp /content/drive/MyDrive/SynToReal/data/train_total/labels/*.txt /content/drive/MyDrive/SynToReal/yolo_dataset/labels/train/
